# 날짜를 파일명에 추가해서 저장, DB에 저장

In [1]:
import time
from datetime import date
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
today = date.today()
print(today, type(today))
today = str(today)
# print(today, type(today))

2024-08-28 <class 'datetime.date'>


In [3]:
page = 1
page_size = 100
final_result_df = pd.DataFrame()
while True:
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method='searchCorpList',pageIndex=page, currentPageSize=page_size,orderMode=3,orderStat='D',searchType=13, fiscalYearEnd='all', location='all')
    r = requests.get(url, params=payload)
    print(r.status_code, end="\r")
    soup = bs(r.text, 'lxml')
    total_items = int(soup.select_one(".info.type-00 > em").text.replace(",", ""))
    total_pages = total_items // page_size + 1
    print(f"{page}/{total_pages} 수집중", end="\r")
    keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")  
    result = {}
    for tr in soup.select('tr'):
        for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
            if idx == 0:
                kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
                kind = ", ".join(kinds)
                code = td.select_one('a')['onclick'].split("'")[1]+"0" # 종목코드 추출
                result.setdefault('증권종류', []).append(kind) # 증권종류 저장
                result.setdefault(key, []).append(td.text)   # 회사이름 저장
                result.setdefault('종목코드', []).append(code)
            elif idx == 6:
                home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
                result.setdefault(key, []).append(home_link)
            else:
                result.setdefault(key, []).append(td.text)
    result_df = pd.DataFrame(result)
    final_result_df = pd.concat([final_result_df, result_df])
        
    if page < total_pages:
        page += 1
        time.sleep(5)
    else:
        break

display(final_result_df)
# final_result_df.to_csv(f"상장기업정보{today}_기준.csv", encoding='utf-8', index=False)  # 수집일 기준으로 csv 파일로 저장



,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,이엔셀,456070,기초 의약물질 제조업,"첨단바이오의약품 위탁개발생산(CDMO) 서비스 / 차세대 세포,유전자 치료제",2024-08-23,12월,장종욱,http://www.encellinc.com/,서울특별시
1,코스닥,M83,476080,"영화, 비디오물, 방송프로그램 제작 및 배급업",VFX,2024-08-22,12월,정성진..,http://m83.co.kr,서울특별시
2,코스닥,대신밸런스제18호스팩,478780,금융 지원 서비스업,금융 지원 서비스업,2024-08-22,12월,문병식,,서울특별시
3,코스닥,티디에스팜,464280,의약품 제조업,"경피약물전달시스템(습포제, 첩부제)",2024-08-21,12월,김철준,http://www.tdspharm.com,경기도
4,코스닥,넥스트바이오메디컬,389650,의료용 기기 제조업,"내시경용 지혈재, 혈관색전 미립구",2024-08-20,12월,이돈행,http://www.nextbiomedical.co.kr,인천광역시
...,...,...,...,...,...,...,...,...,...,...
11,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
12,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
13,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
14,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [4]:
final_result_df.columns

Index(['증권종류', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지',
       '지역'],
      dtype='object')

In [5]:
final_result_df = pd.read_csv("./상장기업정보2024-07-29_기준.csv")
final_result_df

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,산일전기,062040,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","유입, 몰드, 주상, 건식 변압기 등",2024-07-29,12월,박동석,http://www.sanil.co.kr,경기도
1,유가증권,에이치에스효성,487570,기타 금융업,지주사업,2024-07-29,12월,조현상..,http://www.hshyosung.com,서울특별시
2,"코스닥, 투자주의종목",엔에이치스팩31호,481890,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,NaN,서울특별시
3,코스닥,SK증권제13호스팩,473950,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,NaN,서울특별시
4,코스닥,엑셀세라퓨틱스,373110,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,NaN,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2700,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2701,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2702,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2703,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


# 기업정보 DB에 저장하기

In [6]:
# !conda install PyMySQL       # mysql에 접속하게 해주는 라이브러리
# !conda install sqlalchemy    # sql을 쉽게 사용하게 해주는 라이브러리

In [7]:
import os
from sqlalchemy import create_engine, text
import pymysql
pymysql.install_as_MySQLdb()
from dotenv import load_dotenv
load_dotenv(dotenv_path=".dbenv")

True

In [16]:
# SQLAlchemy 엔진 생성 => mysql 서버에 접속
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('user')}:{os.getenv('pw')}@59.7.246.88:3306")
conn = engine.connect()

In [17]:
# 데이터베이스에 접속 후 데이터베이스 생성
conn.execute(text('CREATE DATABASE IF NOT EXISTS korea_stock_info'))
conn.close()

In [18]:
today = today.replace("-", "_")
today

'2024_08_28'

In [19]:
# !pip install cryptography

In [21]:
#DataFrame을 MySQL 테이블로 저장
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('user')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()
final_result_df.to_sql(f'stock_company_info', con=conn, if_exists='replace', index=False)
conn.close()

In [22]:
final_result_df

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,산일전기,062040,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","유입, 몰드, 주상, 건식 변압기 등",2024-07-29,12월,박동석,http://www.sanil.co.kr,경기도
1,유가증권,에이치에스효성,487570,기타 금융업,지주사업,2024-07-29,12월,조현상..,http://www.hshyosung.com,서울특별시
2,"코스닥, 투자주의종목",엔에이치스팩31호,481890,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,NaN,서울특별시
3,코스닥,SK증권제13호스팩,473950,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,NaN,서울특별시
4,코스닥,엑셀세라퓨틱스,373110,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,NaN,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2700,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2701,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2702,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2703,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [ ]:
keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")

In [ ]:
result = {}
for tr in soup.select('tr'):
    for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
        if idx == 0:
            kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
            kind = ", ".join(kinds)
            code = td.select_one("a")['onclick'].split("'")[1] # 종목코드 추출
            result.setdefault('증권종류', []).append(kind)
            result.setdefault(key, []).append(td.text)
            result.setdefault('종목코드', []).append(code)
        elif idx == 6:
            home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
            result.setdefault(key, []).append(home_link)
        else:
            result.setdefault(key, []).append(td.text)

        
result
result_df = pd.DataFrame(result)
result_df            
       



# pandas read_html로 표 데이터 가져오기

In [ ]:
page = 1
page_size = 100
final_result_df = pd.DataFrame()

url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method='searchCorpList',pageIndex=page, currentPageSize=page_size,orderMode=3,orderStat='D',searchType=13, fiscalYearEnd='all', location='all')
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.text, 'lxml')

In [ ]:
soup.select('tr')[1]

In [ ]:
soup.select('tr')[1].select_one('a')['onclick'].split("'")[1] +"0"

In [ ]:
# 종목코드
for tr in soup.select('tr')[1:2]:
    for td in soup.select('td')[:1]:
        print(td.select_one('a')['onclick'].split("'")[1]+"0")

In [ ]:
df = pd.read_html(r.text)
df[0]